<!-- dom:TITLE: Ordinary Differential Equations (ODE) -->
# Ordinary Differential Equations (ODE)
<!-- dom:AUTHOR: Prepared as part of MOD510 Computational Engineering and Modeling -->
<!-- Author: -->  
**Prepared as part of MOD510 Computational Engineering and Modeling**

Date: **Nov 22, 2019**

# ODE Notebook
Learning objectives:
* being able to implement a numerical algorithm in python

* quantify numerical uncertainty

* test different methods and have basic understanding of the strength and weaknesses of each method

<!-- --- begin exercise --- -->

## Exercise 1: Runge-Kutta Methods

<!-- FIGURE: [fig-ode/rk_fig, width=800] Illustration of the Euler algorithm, and a motivation for using the slope a distance from the $t_n$.<div id="fig:ode:rk"></div> -->

The Euler method only have an accuracy of order $h$, and a global error that do not go to zero as the step size decrease. The 2. order Runge-Kutta method is accurate to $h^2$. 
### The 2. order Runge-Kutta:

$$
k_1=hf(y_n,t_n)\nonumber
$$

$$
k_2=hf(y_n+\frac{1}{2}k_1,t_n+h/2)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:rk4"></div>

$$
\begin{equation}  
y_{n+1}=y_n+k_2\label{eq:ode:rk4} \tag{1}
\end{equation}
$$

### The 4. order Runge-Kutta:

The Runge-Kutta fourth order method is one of he most used methods, it is accurate to order $h^4$, and has an error of order $h^5$.

$$
k_1=hf(y_n,t_n)\nonumber
$$

$$
k_2=hf(y_n+\frac{1}{2}k_1,t_n+h/2)\nonumber
$$

$$
k_3=hf(y_n+\frac{1}{2}k_2,t_n+h/2)\nonumber
$$

$$
k_4=hf(y_n+k_3,t_n+h)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:rk5"></div>

$$
\begin{equation}  
y_{n+1}=y_n+\frac{1}{6}(k_1+2k_2+2k_3+k_4)\label{eq:ode:rk5} \tag{2}
\end{equation}
$$

In the following we are going to solve the CSTR equation:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr2"></div>

$$
\begin{equation}
V\frac{dC(t)}{dt} = q(t)\left[C_\text{in}(t) - C(t)\right].\label{eq:ode:cstr2} \tag{3}
\end{equation}
$$

Seawater contains about 35 gram salt/liter fluid, if we assume that the fresh water contains no salt, we have the boundary conditions $C_\text{in}(t)=0$, $C(0)=$35gram/l. 
Complete the code below, and test it both with the Runge-Kutta 2. and 4. order method:

In [1]:
%matplotlib inline


import matplotlib.pyplot as plt
import numpy as np

def fm(c_old,c_in):
    return ...

def rk4_step(c_old, c_in, h):
    ... 
    return

def rk2_step(c_old, c_in, h):
    ... 
    return 
# The following code is a suggestion, most likely it can be done more efficiently
def ode_solv(c_into,c_init,t_final,h):
    f=[];t=[]
    c_in  = c_into #freshwater into tank
    c_old = c_init #seawater present 
    ti=0.
    while(ti <= t_final):
        t.append(ti); f.append(c_old)
        c_new = rk4_step(c_old,c_in,h) # or rk2_step    
        c_old = c_new
        ti   += h
    return t,f
# rest of code is to make a figure
# choose some step sizes to investigate
h = [0.4,0.8,1.6,2.8]
# initial values
vol=1000;q=1;c_into = 0; c_init = 35
tau=vol/q;t_final=10 # end of simulation in min
t=[];f=[]
for dti in h:
    ti,fi = ode_solv(c_into,c_init,t_final,dti)
    t.append(np.array(ti));f.append(fi)
f_an = c_init*np.exp(-np.array(t[0]))
symb = ['-p','-v','-*','-s']
fig = plt.figure(dpi=150)
for i in range(0,len(h)):
    plt.plot(tau*t[i], f[i], symb[i], label='$\Delta t =$'
             +str(h[i]*tau)+" min")
plt.plot(tau*t[0], f_an, '-', color='k',label='analytical')
plt.legend(loc='upper right', ncol=1)
plt.ylim([0,50])
plt.grid()
plt.xlabel('Time [min]')
plt.ylabel('Concentration')
plt.show()

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 2: Adaptive step size - Runge-Kutta Method

In this exercise you are going to improve the algorithms above by choosing a step size that is not too large or too small. This will *greatly* enhance the efficiency of the code. We are going to use the following result from the compendium (to get a good understanding it is advised to derive them, see the compendium [[hiorth]](#hiorth))

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
|\epsilon|=\frac{|\Delta|}{2^p-1}=\frac{|y_1^*-y_1|}{2^p-1},
\label{_auto1} \tag{4}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}  
h^\prime=\beta h\left|\frac{\epsilon}{\epsilon_0}\right|^{\frac{1}{p+1}},
\label{_auto2} \tag{5}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation}  
\hat{y_1}=y_1-\epsilon=\frac{2^p y_1-y_1^*}{2^{p}-1},
\label{_auto3} \tag{6}
\end{equation}
$$

where $\beta$ is a safety factor $\beta\simeq0.8,0.9$, and you should always be careful that the step size do not become too large so that
the method breaks down. This can happens when $\epsilon$ is very low, which may happen if $y_1^*\simeq y_1$ and/or if $y_1^*\simeq y_1\simeq 0$.  

Use the equations above, and implement an adaptive step size algorithm for the 2. and 4. order Runge-Kutta methods. Use the CSTR reactor, and the seawater case as above to test your implementation.  It might be a good idea to use a safety limit on the step size 'min(hi*(tol/toli)**(0.2),1)'.

In [2]:
import matplotlib.pyplot as plt
import numpy as np

def adaptive_ode_solv(c_into,c_init,t_final,tol=1e-4):
    f=[];t=[]
    tau_inv = q/vol
    c_in    = c_into #freshwater into tank
    c_old   = c_init #seawater present 
    ti=0.; h_new=1;
    toli=1.; # a high init tolerance to enter while loop
    no_steps=0
    global_err=0.
    while(ti <= t_final):
        t.append(ti); f.append(c_old)
        tol = tol + tol*np.fabs(c_old) 
        while(toli>tol):
            hi=h_new
	    # first two small steps:
            k1 = rk4_step(c_old,c_in, ...)
            k2 = rk4_step(k1,c_in, ...)
            # ... and one large step
            k3 = rk4_step(c_old,c_in,...)
	    # estimate local error
            toli = ... (equation above)
	    # new step size
            h_new=min(... (equation above) ... ,1)
            no_steps+=3
        toli=1.
	# estimate new solution 
        c_old= ... (equation above)... 
        ti   += hi
    print("No steps=", no_steps)
    return t,f
# rest of code is to make a figure
vol=1;q=1;c_into = 0; c_init =1
t_final=10 # end of simulation
t=[];f=[]
tol=[1e-8,1e-7,1e-6,1e-5]
for toli in tol:
    ti,fi = adaptive_ode_solv(c_into,c_init,t_final,toli)
    t.append(ti);f.append(fi)

f_an = np.exp(-np.array(t[0]))
symb = ['-p','-v','-*','-s']
fig = plt.figure(dpi=150)
for i in range(0,len(tol)):
    plt.plot(t[i], f[i], symb[i], label='tol='+str(tol[i]))
plt.plot(t[0], f_an, '-', color='k',label='analytical')
plt.legend(loc='lower left', ncol=1)
plt.grid()
plt.yscale('log')
plt.xlabel(r'Time [min/$\tau$]')
plt.ylabel('Concentration')
plt.show()

* How many steps do you need for the Runge-Kutta of 2. order compared to 4. order

* The tolerance might be calculated as $\epsilon^\prime = atol +|y|rtol$, where 'atol' is the absolute tolerance and 'rtol' is the relative tolerance. A sensible choice would be to set 'atol=rtol' (e.g. = $10^{-4}$).

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3: Solving a set of ODE equations

What happens if we have more than one equation that needs to be solved? If we continue with our current example, we might be interested in what would happen 
if we had multiple tanks in series. This could be a very simple model to describe the cleaning  of a salty lake by injecting fresh water into it, but at 
the same time this lake was connected to two nearby fresh water lakes, as illustrated in [figure](#fig:ode:cstr3). The weakest part of the model is the assumption about 
complete mixing, in a practical situation we could enforce complete mixing with the salty water in the first tank by injecting fresh water at multiple point in the 
lake. For the two next lakes, the degree of mixing is not obvious, but salt water is heavier than fresh water and therefore it would sink and mix with the fresh water. Thus
if the flow rate was slow, one might imaging that a more or less complete mixing could occur. Our model then could answer questions like, how long time would it take before most
of the salt water is removed from the first lake, and how much time would it take before most of the salt water was cleared from the whole system? The answer to 
these questions would give practical input on how much and how fast one should inject the fresh water to clean up the system. If we had 
data from an actual system, we could compare our model predictions with data from the physical system, and investigate if our model description was correct. 

<!-- dom:FIGURE: [fig-ode/cstr3.png, width=800] A simple model for cleaning a salty lake that is connected to two lakes down stream. <div id="fig:ode:cstr3"></div> -->
<!-- begin figure -->
<div id="fig:ode:cstr3"></div>

<p>A simple model for cleaning a salty lake that is connected to two lakes down stream.</p>
<img src="fig-ode/cstr3.png" width=800>

<!-- end figure -->


For simplicity we will assume that all the lakes have the same volume, $V$. The governing equations follows
as before, by assuming mass balance:

$$
C_0(t+\Delta t)\cdot V - C_0(t)\cdot V = q(t)\cdot C_\text{in}(t)\cdot \Delta t - q(t)\cdot C_0(t)\cdot \Delta t,\nonumber
$$

$$
C_1(t+\Delta t)\cdot V - C_1(t)\cdot V = q(t)\cdot C_0(t)\cdot \Delta t - q(t)\cdot C_1(t)\cdot \Delta t,\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3a"></div>

$$
\begin{equation}  
C_2(t+\Delta t)\cdot V - C_2(t)\cdot V = q(t)\cdot C_1(t)\cdot \Delta t - q(t)\cdot C_2(t)\cdot \Delta t.\label{eq:ode:cstr3a} \tag{7}
\end{equation}
$$

Taking the limit $\Delta t\to 0$, we can write equation ([7](#eq:ode:cstr3a)) as:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3b"></div>

$$
\begin{equation}
V\frac{dC_0(t)}{dt} = q(t)\left[C_\text{in}(t) - C_0(t)\right],\label{eq:ode:cstr3b} \tag{8}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3c"></div>

$$
\begin{equation}  
V\frac{dC_1(t)}{dt} = q(t)\left[C_0(t) - C_1(t)\right],\label{eq:ode:cstr3c} \tag{9}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3d"></div>

$$
\begin{equation}  
V\frac{dC_2(t)}{dt} = q(t)\left[C_1(t) - C_2(t)\right].\label{eq:ode:cstr3d} \tag{10}
\end{equation}
$$

Show that the analytical solution is:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3h"></div>

$$
\begin{equation}
V\frac{dC_2(t)}{dt} = q(t)\left[\frac{C_{0,0}t}{\tau}e^{-t/\tau} - C_2(t)\right],\label{eq:ode:cstr3h} \tag{11}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3i"></div>

$$
\begin{equation}  
\frac{d}{dt}\left[e^{t/\tau}C_2\right]= \frac{C_{0,0}t}{\tau},\label{eq:ode:cstr3i} \tag{12}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3j"></div>

$$
\begin{equation}  
C_2(t)=\frac{C_{0,0}t^2}{2\tau^2}e^{-t/\tau}.\label{eq:ode:cstr3j} \tag{13}
\end{equation}
$$

The numerical solution follows the exact same pattern as before if we introduce a vector notation. Before doing that, we rescale the time $t\to t/\tau$ and the concentrations,
 $\hat{C_i}=C_i/C_{0,0}$ for $i=0,1,2$, hence:

$$
\frac{d}{dt}
\left(
\begin{array}{c} 
 \hat{C_0}(t)\\ 
 \hat{C_1}(t)\\ 
 \hat{C_2}(t)
 \end{array}
 \right)
=\left(
\begin{array}{c} 
 \hat{C_\text{in}}(t) - \hat{C_0}(t)\\ 
 \hat{C_0}(t) - \hat{C_1}(t)\\ 
 \hat{C_1}(t) - \hat{C_2}(t)
 \end{array}
 \right),\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation}  
 \frac{d\mathbf{\hat{C}}(t)}{dt}=\mathbf{f}(\mathbf{\hat{C}},t).
\label{_auto4} \tag{14}
\end{equation}
$$

Finnish the implementation of the code:

In [3]:
import matplotlib.pyplot as plt
import numpy as np

def fm(c_old,c_in,tau):
    return (c_in-c_old)/tau

def rk4_step(c_old, c_in, tau, h):
    #vectorize the code
    return c_next

def ode_solv(c_into,c_init,t_final,tau,h):
    f=[];t=[]
    c_in  = c_into #freshwater into first tank
    c_old = c_init #seawater present 
    ti=0.
    while(ti <= t_final):
        t.append(ti); f.append(c_old)
        c_new = rk4_step(c_old,c_in,tau,h)     
        c_old = c_new
        # add boundary condition - put concentration of tank 0 into tank 1 etc.

        ti   += h
    return np.array(t),np.array(f)
h = 1e-2
# initial values
vol=1;q=1;c_into = [0,0,0]; c_init = [1,0,0]
tau=[1,1,1];t_final=10 # end of simulation 
t,f = ode_solv(c_into,c_init,t_final,tau,h)
# rest of code is to make a figure

f_an = []
f_an.append(c_init[0]*np.exp(-t))
f_an.append(c_init[0]*t*np.exp(-t))
f_an.append(c_init[0]*0.5*t*t*np.exp(-t))

symb = ['-p','-v','-*','-s']
fig = plt.figure(dpi=150)
for i in range(0,len(c_init)):
    legi = '$\hat{C}_'+str(i)+'(\\tau)$'
    plt.plot(t, f[:,i], '-', label=legi,lw=4)
    plt.plot(t, f_an[i], '--', color='k')
plt.plot(0,0 , '--', color='k',label='analytical')
plt.legend(loc='upper right', ncol=1)
#plt.ylim([0,50])
plt.grid()
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.show()

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 4: Stiff sets of ODE  and implicit methods

As already mentioned a couple of times, our system could be part of a much larger system. To illustrate this, let us now assume that we have two 
tanks in series. The first tank is similar to our original tank, but the second tank is a sampling tank, 1000 times smaller.   

<!-- dom:FIGURE: [fig-ode/cstr2.png, width=800] A continuous stirred tank model with a sampling vessel. <div id="fig:ode:cstr2"></div> -->
<!-- begin figure -->
<div id="fig:ode:cstr2"></div>

<p>A continuous stirred tank model with a sampling vessel.</p>
<img src="fig-ode/cstr2.png" width=800>

<!-- end figure -->


The governing equations can be found by requiring mass balance for each of the tanks:

$$
C_0(t+\Delta t)\cdot V_0 - C_0(t)\cdot V_0 = q(t)\cdot C_\text{in}(t)\cdot \Delta t - q(t)\cdot C_0(t)\cdot \Delta t.\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr2a"></div>

$$
\begin{equation}  
C_1(t+\Delta t)\cdot V_1 - C_1(t)\cdot V_1 = q(t)\cdot C_0(t)\cdot \Delta t - q(t)\cdot C_1(t)\cdot \Delta t.
\label{eq:ode:cstr2a} \tag{15}
\end{equation}
$$

Taking the limit $\Delta t\to 0$, we can write equation ([15](#eq:ode:cstr2a)) as:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr2bb"></div>

$$
\begin{equation}
V_0\frac{dC_0(t)}{dt} = q(t)\left[C_\text{in}(t) - C_0(t)\right].\label{eq:ode:cstr2bb} \tag{16}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr2b"></div>

$$
\begin{equation}  
V_1\frac{dC_1(t)}{dt} = q(t)\left[C_0(t) - C_1(t)\right].\label{eq:ode:cstr2b} \tag{17}
\end{equation}
$$

Assume that the first tank is filled with seawater, $C_0(0)=C_{0,0}$, and fresh water is flooded into the tank, i.e. $C_\text{in}=0$. Before we start to consider a numerical
solution, let us first find the analytical solution: As before the solution for the first tank (equation ([16](#eq:ode:cstr2bb))) is:

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation}
C_0(t)=C_{0,0}e^{-t/\tau_0},
\label{_auto5} \tag{18}
\end{equation}
$$

where $\tau_0\equiv V_0/q$. Inserting this equation into equation ([17](#eq:ode:cstr2b)), we get:

$$
\frac{dC_1(t)}{dt} = \frac{1}{\tau_1}\left[C_{0,0}e^{-t/\tau_0} - C_1(t)\right],\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr2c"></div>

$$
\begin{equation}  
\frac{d}{dt}\left[e^{t/\tau_2}C_1\right]= \frac{C_{0,0}}{\tau_1}e^{-t(1/\tau_0-1/\tau_1)}\label{eq:ode:cstr2c} \tag{19},
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr2d"></div>

$$
\begin{equation}  
C_1(t)=\frac{C_{0,0}}{1-\frac{\tau_1}{\tau_0}}\left[e^{-t/\tau_0}-e^{-t/\tau_1}\right],\label{eq:ode:cstr2d} \tag{20}
\end{equation}
$$

where $\tau_1\equiv V_1/q$.

The methods we have considered so far are known as *explicit*, whenever we replace the solution in the right hand side of our algorithm with $y(t+\Delta t)$
or ($y_{n+1}$),
the method is known as *implicit*. Implicit methods are always stable, meaning that we can take as large a time step that we would like, without
getting oscillating solution. 

* Show that:

$$
{C_0}_{n+1}=\frac{{C_0}_n + \frac{\Delta t}{\tau_0}{C_\text{in}}_{n+1}}{1+\frac{\Delta t}{\tau_0}},\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstri1"></div>

$$
\begin{equation}  
{C_2}_{n+1}=\frac{{C_1}_n + \frac{\Delta t}{\tau_1}{C_0}_{n+1}}{1+\frac{\Delta t}{\tau_1}}.\label{eq:ode:cstri1} \tag{21}
\end{equation}
$$

* Finnish the implementation below

In [4]:
import matplotlib.pyplot as plt
import numpy as np

def fm(c_old,c_in,tau,h):
    return ....

def euler_step(c_old, c_in, tau, h):
    c_next=[]
    for i in range(len(c_old)):
        if(i>0): c_in[i]=c_next[i-1] # c_new in next tank
        c_next.append(fm(c_old[i],c_in[i],tau[i],h))
    return c_next

def ode_solv(c_into,c_init,t_final,tau,h):
    f=[];t=[]
    c_in  = c_into #freshwater into first tank
    c_old = c_init #seawater present 
    ti=0.
    while(ti <= t_final):
        t.append(ti); f.append(c_old)
        c_new = euler_step(c_old,c_in,tau,h)     
        c_old = c_new
        # put concentration of tank 0 into tank 1 etc.
        # ...
        ti   += h
    return np.array(t),np.array(f)
h = 0.01
# initial values
vol=1;q=1;c_into = [0,0]; c_init = [1,0]
tau=[1,1e-3];t_final=10 # end of simulation 
t,f = ode_solv(c_into,c_init,t_final,tau,h)
# rest of code is to make a figure

f_an = [];t_an=np.arange(0,t_final,0.01)
f_an.append(c_init[0]*np.exp(-t_an))
f_an.append(c_init[0]*(1-tau[1]/tau[0])*(np.exp(-t_an/tau[0])-np.exp(-t_an/tau[1])))
symb = ['-p','-v','-*','-s']
fig = plt.figure(dpi=150)
for i in range(0,len(c_init)):
    legi = '$\hat{C}_'+str(i)+'(\\tau)$'
    plt.plot(t, f[:,i], '-', label=legi,lw=4)
    plt.plot(t_an, f_an[i], '--', color='k')
plt.plot(0,0 , '--', color='k',label='analytical')
plt.legend(loc='upper right', ncol=1)
#plt.ylim([0,50])
plt.grid()
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.show()

* Do you need more or less step to reach the same accuracy with the implicit method, compared to Eulers (explicit) method?

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 5: Truncation Error in Eulers Method

We know that Eulers algorithm is accurate to second order. Our estimate of the new value, $y_1^*$  
(where we have used a$\,{}^*$ to indicate that we have used a step size of size $h$), should then be related to the true solution $y(t_1)$ in the following way:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:aeb0"></div>

$$
\begin{equation}
y^*_1=y(t_1)+ch^2.\label{eq:ode:aeb0} \tag{22}
\end{equation}
$$

The constant $c$ is unknown, but it can be found by taking two smaller steps of size $h/2$. If the steps are not too large, our new estimate
of the value $y_1$ will be related to the true solution as:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:aeb1"></div>

$$
\begin{equation}
y_1=y(t_1)+2c\left(\frac{h}{2}\right)^2.\label{eq:ode:aeb1} \tag{23}
\end{equation}
$$

In the following we will take a closer look at the adaptive Eulers algorithm and show that the 
constant $c$ is indeed the same in equation ([22](#eq:ode:aeb0)) and ([23](#eq:ode:aeb1)). 
The true solution $y(t)$, obeys the following equation:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ay"></div>

$$
\begin{equation}
\frac{dy}{dt}=f(y,t),\label{eq:ode:ay} \tag{24}
\end{equation}
$$

and Eulers method to get from $y_0$ to $y_1$ by taking one (large) step, $h$ is:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ae0"></div>

$$
\begin{equation}
y^*_1=y_0+hf(y_0,t_0),\label{eq:ode:ae0} \tag{25}
\end{equation}
$$

We will also assume (for simplicity) that in our starting point $t=t_0$, the numerical solution, $y_0$, is equal to the true solution, $y(t_0)$, hence $y(t_0)=y_0$.


**a)**
Show that when we take one step of size $h$ from $t_0$ to $t_1=t_0+h$, $c=y^{\prime\prime}(t_0)/2$ in equation ([22](#eq:ode:aeb0)).


<!-- --- begin answer of exercise --- -->
**Answer.**
The local error, is the difference between the numerical solution and the true solution:

$$
\epsilon^*=y(t_0+h)-y_{1}^*=y(t_0)+y^{\prime}(t_0)h+\frac{1}{2}y^{\prime\prime}(t_0)h^2+\mathcal{O}(h^3)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto6"></div>

$$
\begin{equation}  
-\left[y_0+hf(y_0,t_0+h)\right],
\label{_auto6} \tag{26}
\end{equation}
$$

where we have used Taylor expansion to expand the true solution around $t_0$, and equation ([25](#eq:ode:ae0)).
Using equation ([24](#eq:ode:ay)) to replace $y^\prime(t_0)$ with $f(y_0,t_0)$, we find:

<!-- Equation labels as ordinary links -->
<div id="_auto7"></div>

$$
\begin{equation}
\epsilon^*=y(t_0+h)-y_{1}^*=\frac{1}{2}y^{\prime\prime}(t_0)h^2\equiv ch^2,
\label{_auto7} \tag{27}
\end{equation}
$$

hence $c=y^{\prime\prime}(t_0)/2$.

<!-- --- end answer of exercise --- -->

**b)**
Show that when we take two steps of size $h/2$ from $t_0$ to $t_1=t_0+h$, Eulers algorithm is:

<!-- Equation labels as ordinary links -->
<div id="_auto8"></div>

$$
\begin{equation}
y_{1}=y_{0}+\frac{h}{2}f(y_0,t_0)+\frac{h}{2}f(y_0+\frac{h}{2}f(y_0,t_0),t_0+h/2).
\label{_auto8} \tag{28}
\end{equation}
$$

<!-- --- begin answer of exercise --- -->
**Answer.**

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ae1b"></div>

$$
\begin{equation}
y_{1/2}=y_0+\frac{h}{2}f(y_0,t_0),\label{eq:ode:ae1b} \tag{29}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ae2b"></div>

$$
\begin{equation}  
y_{1}=y_{1/2}+\frac{h}{2}f(y_{1/2},t_0+h/2),\label{eq:ode:ae2b} \tag{30}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ae3b"></div>

$$
\begin{equation}  
y_{1}=y_{0}+\frac{h}{2}f(y_0,t_0)+\frac{h}{2}f(y_0+\frac{h}{2}f(y_0,t_0),t_0+h/2).\label{eq:ode:ae3b} \tag{31}
\end{equation}
$$

Note that we have inserted
equation ([29](#eq:ode:ae1b)) into equation ([30](#eq:ode:ae2b)) to arrive at equation ([31](#eq:ode:ae3b)).

<!-- --- end answer of exercise --- -->

**c)**
Find an expression for the local error when using two steps of size $h/2$, and show that the local error is: $\frac{1}{2}ch^2$


<!-- --- begin answer of exercise --- -->
**Answer.**

$$
\epsilon=y(t_0+h)-y_{1}=y(t_0)+y^{\prime}(t_0)h+\frac{1}{2}y^{\prime\prime}(t_0)h^2+\mathcal{O}(h^3)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ay5b"></div>

$$
\begin{equation}  
-\left[y_{0}+\frac{h}{2}f(y_0,t_0)+\frac{h}{2}f(y_0+\frac{h}{2}f(y_0,t_0),t_0+h/2)\right].\label{eq:ode:ay5b} \tag{32}
\end{equation}
$$

This equation is slightly more complicated, due to the term involving $f$ inside the last parenthesis, we can use Taylor expansion to expand it about $(y_0,t_0)$:

$$
f(y_0+\frac{h}{2}f(y_0,t_0),t_0+h/2)=f(y_0,t_0)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ay2b"></div>

$$
\begin{equation}  
+\frac{h}{2}\left[f(y_0,t_0)\left.\frac{\partial f}{\partial y}\right|_{y=y_0,t=t_0}
+\frac{h}{2}\left.\frac{\partial f}{\partial t}\right|_{y=y_0,t=t_0}\right]+\mathcal{O}(h^2).\label{eq:ode:ay2b} \tag{33}
\end{equation}
$$

It turns out that this equation is related to $y^{\prime\prime}(t_0,y_0)$, which can be seen by differentiating equation ([24](#eq:ode:ay)):

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ay3b"></div>

$$
\begin{equation}
\frac{d^2y}{dt^2}=\frac{df(y,t)}{dt}=\frac{\partial f(y,t)}{\partial y}\frac{dy}{dt}+\frac{\partial f(y,t)}{\partial t}
=\frac{\partial f(y,t)}{\partial y}f(y,t)+\frac{\partial f(y,t)}{\partial t}.\label{eq:ode:ay3b} \tag{34}
\end{equation}
$$

Hence, equation ([33](#eq:ode:ay2b)) can be written:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ay4b"></div>

$$
\begin{equation}
f(y_0+\frac{h}{2}f(y_0,t_0),t_0+h/2)=f(y_0,t_0)+\frac{h}{2}y^{\prime\prime}(t_0,y_0),\label{eq:ode:ay4b} \tag{35}
\end{equation}
$$

hence the truncation error in equation ([32](#eq:ode:ay5b)) can finally be written:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ae4b"></div>

$$
\begin{equation}
\epsilon=y(t_1)-y_{1}=\frac{h^2}{4} y^{\prime\prime}(y_0,t_0)=\frac{1}{2}ch^2,\label{eq:ode:ae4b} \tag{36}
\end{equation}
$$

<!-- --- end answer of exercise --- -->









<!-- --- begin solution of exercise --- -->
**Solution.**
The local error, is the difference between the numerical solution and the true solution:

$$
\epsilon^*=y(t_0+h)-y_{1}^*=y(t_0)+y^{\prime}(t_0)h+\frac{1}{2}y^{\prime\prime}(t_0)h^2+\mathcal{O}(h^3)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto9"></div>

$$
\begin{equation}  
-\left[y_0+hf(y_0,t_0+h)\right],
\label{_auto9} \tag{37}
\end{equation}
$$

where we have used Taylor expansion to expand the true solution around $t_0$, and equation ([25](#eq:ode:ae0)).
Using equation ([24](#eq:ode:ay)) to replace $y^\prime(t_0)$ with $f(y_0,t_0)$, we find:

<!-- Equation labels as ordinary links -->
<div id="_auto10"></div>

$$
\begin{equation}
\epsilon^*=y(t_0+h)-y_{1}^*=\frac{1}{2}y^{\prime\prime}(t_0)h^2\equiv ch^2,
\label{_auto10} \tag{38}
\end{equation}
$$

where we have ignored terms of higher order than $h^2$, and defined $c$ as $c=y^{\prime\prime}(t_0)/2$. Next we take two steps of size $h/2$ to
reach $y_1$:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ae1"></div>

$$
\begin{equation}
y_{1/2}=y_0+\frac{h}{2}f(y_0,t_0),\label{eq:ode:ae1} \tag{39}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ae2"></div>

$$
\begin{equation}  
y_{1}=y_{1/2}+\frac{h}{2}f(y_{1/2},t_0+h/2),\label{eq:ode:ae2} \tag{40}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ae3"></div>

$$
\begin{equation}  
y_{1}=y_{0}+\frac{h}{2}f(y_0,t_0)+\frac{h}{2}f(y_0+\frac{h}{2}f(y_0,t_0),t_0+h/2).\label{eq:ode:ae3} \tag{41}
\end{equation}
$$

Note that we have inserted
equation ([39](#eq:ode:ae1)) into equation ([40](#eq:ode:ae2)) to arrive at equation ([41](#eq:ode:ae3)). The truncation error in this case is, as before:

$$
\epsilon=y(t_0+h)-y_{1}=y(t_0)+y^{\prime}(t_0)h+\frac{1}{2}y^{\prime\prime}(t_0)h^2+\mathcal{O}(h^3)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ay5"></div>

$$
\begin{equation}  
-\left[y_{0}+\frac{h}{2}f(y_0,t_0)+\frac{h}{2}f(y_0+\frac{h}{2}f(y_0,t_0),t_0+h/2)\right].\label{eq:ode:ay5} \tag{42}
\end{equation}
$$

This equation is slightly more complicated, due to the term involving $f$ inside the last parenthesis, we can use Taylor expansion to expand it about $(y_0,t_0)$:

$$
f(y_0+\frac{h}{2}f(y_0,t_0),t_0+h/2)=f(y_0,t_0)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ay2"></div>

$$
\begin{equation}  
+\frac{h}{2}\left[f(y_0,t_0)\left.\frac{\partial f}{\partial y}\right|_{y=y_0,t=t_0}
+\left.\frac{\partial f}{\partial t}\right|_{y=y_0,t=t_0}\right]+\mathcal{O}(h^2).\label{eq:ode:ay2} \tag{43}
\end{equation}
$$

It turns out that this equation is related to $y^{\prime\prime}(t_0,y_0)$, which can be seen by differentiating equation ([24](#eq:ode:ay)):

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ay3"></div>

$$
\begin{equation}
\frac{d^2y}{dt^2}=\frac{df(y,t)}{dt}=\frac{\partial f(y,t)}{\partial y}\frac{dy}{dt}+\frac{\partial f(y,t)}{\partial t}
=\frac{\partial f(y,t)}{\partial y}f(y,t)+\frac{\partial f(y,t)}{\partial t}.\label{eq:ode:ay3} \tag{44}
\end{equation}
$$

Hence, equation ([43](#eq:ode:ay2)) can be written:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ay4"></div>

$$
\begin{equation}
f(y_0+\frac{h}{2}f(y_0,t_0),t_0+h/2)=f(y_0,t_0)+\frac{h}{2}y^{\prime\prime}(t_0,y_0),\label{eq:ode:ay4} \tag{45}
\end{equation}
$$

hence the truncation error in equation ([42](#eq:ode:ay5)) can finally be written:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:ae4"></div>

$$
\begin{equation}
\epsilon=y(t_1)-y_{1}=\frac{h^2}{4} y^{\prime\prime}(y_0,t_0)=\frac{1}{2}ch^2,\label{eq:ode:ae4} \tag{46}
\end{equation}
$$

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->


## References

1. <div id="hiorth"></div> **A. Hiorth**. 
    *Computational Engineering and Modeling*,
    https://github.com/ahiorth/CompEngineering,
    2019.